In [5]:
from icecube import icetray,dataio, dataclasses, clsim, simclasses
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [6]:
geofilepath = '/cvmfs/icecube.opensciencegrid.org/data/GCD/GeoCalibDetectorStatus_AVG_55697-57531_PASS2_SPE_withScaledNoise.i3.gz'
geofile = dataio.I3File(geofilepath)
for geoframe in geofile:
    if geoframe.Stop == geoframe.Geometry:
        break
omgeo = geoframe['I3Geometry'].omgeo

In [7]:
class Photon():
    def __init__(self, filepath):
        self.filepath = filepath
        self.frame = None
    
    def extract_frame(self):
        # Extracting the daq frame once#
        if self.frame is None:
            file = dataio.I3File(self.filepath)
            self.frame = file.pop_daq()
        return self.frame

    def generate_time(self, string, module):
        
        if self.frame is None:
            self.extract_frame()
            
        # Getting all the photons #    
        photons = self.frame["I3PhotonSeriesMap"]
        signal = photons.get(dataclasses.ModuleKey(string, module))

        # POCAM Flash #
        self.pulse = self.frame['I3FlasherPulseSeries']
        x_dist = abs(omgeo.get(icetray.OMKey(string,module)).position.x - self.pulse[0].pos.x)

        # Calculating the time # 
        try:
            time = np.array([ signal[i].time for i,j in enumerate(signal) ])
            g_time = np.array([ i - (x_dist/dataclasses.I3Constants.c_ice)
                      for i in time])
            HistogramCounts,_ = np.histogram(g_time, bins = 30)
        except TypeError:
            HistogramCounts = np.zeros(30, dtype=int) 

        return HistogramCounts, string, module



In [17]:
class CreateDatabase(Photon):
    
    def __init__(self,filepath,scattering_filepath):
        super().__init__(filepath)
        self.scattering_filepath = scattering_filepath

    def Data(self):
        database = []
        for st in range(1,87):
            for mod in range(1,61):
                HistogramCounts, String, Module = self.generate_time(st, mod)
                database.append({
                    **{f'bin{i}': HistogramCounts[i] for i in range(30)},
                    "String": String,
                    "Module" : Module,
                    'POCAM_zPostion': 1948.07 - self.pulse[0].pos.z,
                    'Distnace': abs(self.pulse[0].pos.x - omgeo.get(icetray.OMKey(st,mod)).position.x)
                    })
        Data = pd.DataFrame(database)

        scattering_file = pd.read_csv(self.scattering_filepath, sep = ' ', header=None)

        temp_df = []
        for st in range(1,87):
            for mod in range(1,61):
                temp_df.append({
                    'String': st,
                    'Module': mod,
                    'Module_zPosition': 1948.07 - omgeo.get(icetray.OMKey(st, mod)).position.z 
                      })
        temp_df = pd.DataFrame(temp_df)

        # Filter and sort #
        mask = (scattering_file[0] > 1448.47) & (scattering_file[0] < 2458)
        filtered_df = scattering_file[mask].reset_index(drop=True)
        z_values = filtered_df[0].values
        scattering_values = filtered_df[1].values
        # Find insertion indices (nearest lower values)
        indices = np.searchsorted(z_values, temp_df["Module_zPosition"].values, side='right') - 1
        # Clip to valid range #
        indices = np.clip(indices, 0, len(z_values) - 1)
        # Assign scattering values
        temp_df['Scattering'] = scattering_values[indices]

        redundant_col = ['String', 'Module']
        Data = pd.concat([Data, temp_df.drop(columns=redundant_col)], axis=1)

        return Data

        

In [18]:
Database_scaledown20 = CreateDatabase(filepath="/home/amithan/Test-first/POCAM_Flash_ScaleddownBy20.i3.bz2",scattering_filepath='/data/user/amithan/software/icetray_POCAM/src/ice-models/resources/models/ICEMODEL/spice_ftp-v3-scaledown-20/icemodel.dat')
df1 = Database_scaledown20.Data()

In [19]:
Database_scaleup20 = CreateDatabase(filepath='/home/amithan/Test-first/POCAM_Flash_ScaledupBy20.i3.bz2', scattering_filepath="/data/user/amithan/software/icetray_POCAM/src/ice-models/resources/models/ICEMODEL/spice_ftp-v3-scaleup-20/icemodel.dat")
df2 = Database_scaleup20.Data()

In [20]:
Database_scaledown10 = CreateDatabase(filepath='/home/amithan/scripts/Flashscripts/Test-first/xProgScaleddown/POCAM_Flash_scaleddown_xpos-166.25-3.i3.bz2', scattering_filepath="/data/user/amithan/software/icetray_POCAM/src/ice-models/resources/models/ICEMODEL/spice_ftp-v3-scaledown-10-v2/icemodel.dat")
df3 = Database_scaledown10.Data()

In [21]:
Database_scaleup10 = CreateDatabase(filepath='/home/amithan/scripts/Flashscripts/Test-first/xProgScaledUp/POCAM_Flash_scaledup_xpos-166.25-3.i3.bz2', scattering_filepath="/data/user/amithan/software/icetray_POCAM/src/ice-models/resources/models/ICEMODEL/spice_ftp-v3-scaleup-10-v2/icemodel.dat")
df4 = Database_scaleup10.Data()

In [22]:
Database_mean = CreateDatabase(filepath='/home/amithan/Test-first/test_flash.i3.bz2', scattering_filepath='/data/user/amithan/software/icetray_POCAM/src/ice-models/resources/models/ICEMODEL/spice_ftp-v3/icemodel.dat')
df5 = Database_mean.Data()

In [23]:
Training_data = pd.concat([df1,df2,df3,df4,df5], axis=0, ignore_index=True)

In [24]:
print(Training_data)

       bin0  bin1  bin2  bin3  bin4  bin5  bin6  bin7  bin8  bin9  ...  bin26   
0         0     0     0     0     0     0     0     0     0     0  ...      0  \
1         0     0     0     0     0     0     0     0     0     0  ...      0   
2         0     0     0     0     0     0     0     0     0     0  ...      0   
3         0     0     0     0     0     0     0     0     0     0  ...      0   
4         0     0     0     0     0     0     0     0     0     0  ...      0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...    ...   
25795     1     0     0     0     0     0     1     0     0     0  ...      0   
25796     1     0     0     0     0     0     0     0     0     0  ...      0   
25797     0     0     0     0     0     0     0     0     0     0  ...      0   
25798     1     0     0     0     0     0     0     0     0     0  ...      0   
25799     1     0     0     0     0     0     0     0     0     0  ...      0   

       bin27  bin28  bin29 

In [25]:
Training_data.to_csv('/home/amithan/NN/Training_data.csv', index=False)